In [1]:
# Core
import numpy as np
import pandas as pd

import time

In [2]:
# Local imports
import kepler_sieve
from asteroid_skypatch import calc_ast_skypatch, test_skypatch_dir
from asteroid_direction import ast, c, mpd, dpm, prep_ast_block, calc_dir_ast_block
from asteroid_spline import make_spline_ast_dir
from sky_patch import dir2SkyPatchID, SkyPatchID2dir
from db_utils import sp2df, df2db

# Build block of asteroid directions at high resolution

In [3]:
# Asteroid block
n0: int = 1
n1: int = 100

# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
interval_min: int = 15

In [4]:
# Asteroid block
n0: int = 1
n1: int = 3

# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
interval_min: int = 15

In [5]:
t0 = time.time()
df = calc_ast_skypatch(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, interval_min=interval_min)
t1 = time.time()

In [6]:
et = t1 - t0
print(f'Elapsed time with interval={interval_min}: {et:5.2f}')

Elapsed time with interval=15:  0.50


In [7]:
df

,AsteroidID,Segment,SkyPatchID,ux,uy,uz,TimeID_0,TimeID_1
0,1,0,6382172,-0.378601,0.924673,0.040506,83520000,83520135
1,1,1,6382171,-0.379180,0.924433,0.040574,83520135,83520225
2,1,2,6384219,-0.379566,0.924272,0.040619,83520225,83520315
3,1,3,6384218,-0.379952,0.924112,0.040664,83520315,83520495
4,1,4,6384217,-0.380723,0.923790,0.040755,83520495,83520675
...,...,...,...,...,...,...,...,...
33092,2,18884,4488003,-0.138457,0.750757,-0.645905,86399550,86399670
33093,2,18885,4490051,-0.138616,0.751112,-0.645458,86399670,86399700
33094,2,18886,4490050,-0.138656,0.751201,-0.645346,86399700,86399790
33095,2,18887,4492098,-0.138777,0.751467,-0.645010,86399790,86399910


# Test direction at midpoint of interval vs. center of sky patch

In [8]:
# Time at the midpoint of each interval
t_obs_0 = df.TimeID_0.values
t_obs_1 = df.TimeID_1.values
t_obs = (t_obs_0 + t_obs_1)/2.0

In [9]:
# Extract arrays
asteroid_id = df.AsteroidID.values
sky_patch_id = df.SkyPatchID.values

In [10]:
# Spline for asteroid directions vs. time
spline_u = make_spline_ast_dir(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [11]:
# Evaluate splined asteroid directions
u_ast, lt_ast = spline_u(ts=t_obs, asteroid_id=asteroid_id)

In [12]:
# Direction used to build sky patch DataFrame
cols_u = ['ux', 'uy', 'uz']
u_df = df[cols_u].values

In [13]:
du = test_skypatch_dir(name1='dataframe', name2='spline', u1=u_df, u2=u_ast, verbose=True)

Angle Difference: spline vs. dataframe in degrees
*Mean  :    85.070*
 Median:    73.371
 Max   :   179.445


In [44]:
# Direction at the center of each sky patch
N_sp = 1024
u_sky = SkyPatchID2dir(SkyPatchID=sky_patch_id, N=N_sp)

In [45]:
def test_skypatch_dir(name1: str, name2: str, u1: np.ndarray, u2: np.ndarray, verbose: bool=False) -> float:
    """
    Report 
    INPUTS:
        name1: Descriptive name of the first source, e.g. 'spline'
        name2: Descriptive name of the second source, e.g. 'skypatch'
        u1:    Array of directions from source 1; shape Nx3
        u2:    Array of directions from source 2; shape Nx3
        verbose: Whether to report results to console
    """
    # Difference in unit directions
    du = u2 - u1
    du_norm = np.sqrt(np.sum(np.square(du), axis=-1))
    du_deg =  dist2deg(du_norm)

    # Calculate mean, median and max difference in degrees
    du_mean = np.mean(du_deg)
    du_median = np.median(du_deg)
    du_max = np.max(du_deg)

    if verbose:
        print(f'Angle Difference: {name2} vs. {name1} in degrees')
        print(f'*Mean  : {du_mean:9.3f}*')
        print(f' Median: {du_median:9.3f}')
        print(f' Max   : {du_max:9.3f}')

    # Return the difference of direction vectors in seconds of arc
    return du_mean

In [24]:
u_ast[0]

array([ 0.93255028, -0.29471628, -0.20854808])

In [41]:
du = u_ast - u_sky

In [42]:
du

array([[-0.57847284, -0.89696923,  0.24773406],
       [-0.57769985, -0.89665278,  0.24774781],
       [-0.57771376, -0.89661889,  0.24684668],
       ...,
       [-0.11748641,  0.07553611,  0.00161565],
       [-0.11698531,  0.07516902,  0.00156063],
       [-0.11648352,  0.07480191,  0.00150559]])